In [10]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [38]:
# with help from http://planspace.org/20150423-forward_selection_with_statsmodels/
import statsmodels.formula.api as smf

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response, ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < (best_new_score) * 0.9:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response, ' + '.join(selected))
    
    model = smf.ols(formula, data).fit()
    return model

In [12]:
importedResponse=pd.read_pickle('data/ZipcodeMSPPS.pkl')
response=importedResponse[['RegionName','City','State','2000_agg','2010_agg','pct_delta']]
response=response.rename(columns={'RegionName': 'Zip_Code'})

In [13]:
response.head(10)

,Zip_Code,City,State,2000_agg,2010_agg,pct_delta
1,60629,Chicago,IL,98.682400,115.605500,1.171491
2,90650,Norwalk,CA,130.505400,229.691000,1.760011
7,90280,South Gate,CA,130.197000,212.896167,1.635185
10,60618,Chicago,IL,151.892400,265.168433,1.745765
13,91331,Los Angeles,CA,104.836333,195.163200,1.861599
14,08701,Lakewood Township,NJ,74.834500,135.057367,1.804747
15,92335,Fontana,CA,79.024267,118.977400,1.505581
21,90805,Long Beach,CA,123.739133,222.963300,1.801882
24,94565,Pittsburg,CA,117.949467,120.324433,1.020135
27,92336,Fontana,CA,84.988300,127.604600,1.501437


In [14]:
cdd=pd.read_pickle('data/cleaned_demographic_features.pkl')
cdd=cdd.rename(columns={'GEO.id2' : 'Zip_Code'})
print cdd.shape
cdd.head(10)

(33120, 59)


,GEO.id,Zip_Code,GEO.display-label,A001,A002,A003,A004,A005,A006,A007,A008,A009,A010,A011,A012,A013,A014,A015,A016,A017,A018,A019,A020,A021,A022,A023,A024,A025,A026,A027,A028,A029,A030,A031,A032,A033,A034,A035,A036,A037,A038,A039,A040,A041,A042,A043,A044,A045,A046,A047,A048,A049,A050,A051,A052,A053,A054,A055,A056
1,8600000US00601,00601,ZCTA5 00601,18570,12.8,16.2,13.9,11.7,13.3,13.1,10.2,5.7,3.1,48.9,6.6,8.5,6.9,5.4,6.3,6.3,4.9,2.7,1.3,51.1,6.3,7.7,7,6.3,6.9,6.8,5.3,2.9,1.8,93.1,3.1,0.4,0,0,99.5,77.8,5.1,2.6,20.3,9.6,20.3,10.2,3.3,10.1,5.5,39.5,29,84.3,15.7,2.9,0.5,2.84,3.27,7744,2.87
2,8600000US00602,00602,ZCTA5 00602,41520,11.5,15.2,13.5,12.9,14.6,14,10.1,5.7,2.3,49.1,5.9,7.7,6.7,6.4,7,6.8,4.9,2.6,1.1,50.9,5.6,7.5,6.8,6.5,7.6,7.2,5.2,3.1,1.3,86.7,5.3,0.3,0.1,0,99.4,77.6,5.1,2.1,17.4,8,20.5,10.5,3,10,4.9,37.6,25.9,83,17,2.6,1.3,2.76,3.20,18073,2.62
3,8600000US00603,00603,ZCTA5 00603,54689,12.2,14.2,13,13,12.7,12.6,12,7,3.3,48.6,6.4,7.3,6.5,6.4,6,5.8,5.6,3.2,1.4,51.4,5.8,7,6.5,6.6,6.6,6.7,6.4,3.7,2.1,82.9,7.6,0.3,0.2,0,98.5,71.1,4.7,1.7,22,10.9,26.4,12.7,4.1,13.8,6.8,34,31.1,82.5,17.5,3.6,1.5,2.53,3.06,25653,2.51
4,8600000US00606,00606,ZCTA5 00606,6615,12.8,15.1,14.7,12.1,13.1,13.4,10.5,5.5,2.9,49.4,6.6,7.5,7.1,6.1,6.5,6.3,5.2,2.6,1.4,50.6,6.2,7.5,7.6,6,6.6,7,5.4,2.9,1.5,88.9,4.7,0.4,0,0.1,99.4,74.6,6.2,2.7,18.4,8.1,22.8,12.7,4.5,10.1,4.7,37.5,28.4,83.6,16.4,1.7,0.8,2.75,3.24,2877,2.83
5,8600000US00610,00610,ZCTA5 00610,29016,11.8,15.1,12.6,13.4,13.9,12.6,11.2,6.7,2.8,48.6,6.2,7.7,6.4,6.6,6.5,5.9,5.1,3.2,1.1,51.4,5.6,7.5,6.2,6.9,7.4,6.7,6.1,3.5,1.7,82,7.2,0.3,0.1,0,99.2,74.4,4.9,1.9,19.9,9.4,22.8,11,3.4,11.9,5.8,36.5,29.4,85.9,14.1,1.9,0.8,2.67,3.14,12618,2.66
6,8600000US00612,00612,ZCTA5 00612,67010,11.4,14.3,12.5,12.6,13.1,12.2,11.8,7.7,4.2,47.4,6,7.3,6.2,6,6.2,5.5,5.2,3.4,1.7,52.6,5.5,7,6.4,6.7,6.9,6.6,6.7,4.4,2.5,84.3,6.2,0.4,0.1,0,99.1,71.7,4.8,1.8,22.4,10.2,25.6,10.7,3.8,14.8,7.8,33.8,33.7,84,16,3.1,2.2,2.55,3.05,30992,2.54
7,8600000US00616,00616,ZCTA5 00616,11017,11.8,14.9,11.8,13.3,14.5,11.4,11.9,6.6,3.7,48.3,6.1,7.5,5.9,6.1,6.9,5.2,5.5,3.2,1.6,51.7,5.7,7.3,5.9,7.2,7.6,6.2,6.4,3.4,2.1,84.8,7.1,0.3,0,0,99.5,73.3,4.4,1.5,20.5,9.1,24.3,12,5,12.2,6.2,36,31.6,84.6,15.4,2.3,1,2.65,3.16,4896,2.53
8,8600000US00617,00617,ZCTA5 00617,24597,13.4,15.1,13,15,12.7,10.9,10.6,6.1,3.2,48.1,6.8,7.8,6.3,7,5.9,5.1,5,2.8,1.3,51.9,6.5,7.4,6.7,8,6.9,5.7,5.5,3.3,1.8,80.6,7.7,0.3,0.1,0,99.4,74.7,5.2,2.2,23.6,11.5,22.8,11.4,3.9,11.4,5.5,38.9,29,85.7,14.3,1.9,2.5,2.71,3.18,10594,2.80
9,8600000US00622,00622,ZCTA5 00622,7853,9.6,13,11,11.1,12.5,13.1,14.6,9.7,5.3,48.2,4.9,6.5,5.2,5.3,6.4,6.1,7.1,4.6,2.1,51.8,4.7,6.5,5.8,5.8,6.2,7.1,7.6,5,3.2,85.8,5,0.2,0,0,98.2,69.1,5.5,2.6,15.5,5.9,26.9,13.4,4.2,13.4,7.3,28.9,37.8,36.6,63.4,3.7,4.9,2.45,2.97,8714,2.51
10,8600000US00623,00623,ZCTA5 00623,43061,12.3,14.4,11.3,14.1,13.3,11.3,11.7,7.5,4,47.8,6.4,7.3,5.6,6.6,6.4,5.2,5.4,3.5,1.6,52.2,6.1,7.1,5.7,7.6,6.9,6.1,6.3,4,2.3,83.8,5.5,0.3,0.1,0,99.1,71.6,4.9,2.2,19.3,9.4,25,11.5,4,13.5,7.3,35.7,32.6,77.6,22.4,1.8,2.7,2.58,3.08,21426,2.61


In [22]:
merged=pd.merge(response, cdd, on=['Zip_Code'])
merged=merged.drop(['Zip_Code','City','State','2000_agg','2010_agg','GEO.id','GEO.display-label'], 1)
merged.head(10)

,pct_delta,A001,A002,A003,A004,A005,A006,A007,A008,A009,A010,A011,A012,A013,A014,A015,A016,A017,A018,A019,A020,A021,A022,A023,A024,A025,A026,A027,A028,A029,A030,A031,A032,A033,A034,A035,A036,A037,A038,A039,A040,A041,A042,A043,A044,A045,A046,A047,A048,A049,A050,A051,A052,A053,A054,A055,A056
0,1.171491,113916,18.1,17.7,16,15.4,12.5,10.2,5.5,2.6,1.8,49.4,9.2,8.9,8.1,7.8,6.2,4.9,2.6,1.1,0.7,50.6,8.9,8.9,8,7.6,6.3,5.3,3,1.5,1.2,41,23,0.7,0.5,0,67.2,78.1,8.7,4.1,22.6,12,18.1,8.7,2.3,9.4,4.1,53.6,20.1,88.6,11.4,4.3,1.7,3.77,4.24,34000,3.21
1,1.760011,105549,14.2,16.8,15.2,14.3,13.6,11.7,7.1,4.1,2.7,49.6,7.4,8.8,7.9,7.1,6.8,5.6,3.3,1.7,1,50.4,6.9,8,7.3,7.2,6.8,6.1,3.9,2.4,1.7,49.4,4.4,1.1,12,0.4,70.1,83.2,8.7,3.6,18.6,7.8,12.6,5.3,1.7,7.3,4.3,50.4,28,96.6,3.4,1.3,0.9,3.83,4.10,28083,3.57
2,1.635185,94396,16.7,18.1,16.2,14.9,13.1,10.4,6.2,2.9,1.6,49.1,8.5,9,8.1,7.4,6.6,4.8,2.8,1.2,0.6,50.9,8.2,9.1,8.1,7.5,6.5,5.5,3.3,1.7,1,50.5,0.9,0.9,0.8,0.1,94.8,86.6,9.7,5,20.2,10.7,9.8,4.5,1.3,5.4,3,59.3,21.9,96.3,3.7,1.9,0.7,4.05,4.24,24160,3.78
3,1.745765,92084,14.5,10.4,19,20.7,13.8,10.2,6.3,3.3,1.9,50.1,7.2,5.3,9.5,10.7,7.3,5.1,2.9,1.3,0.7,49.9,7.2,5.1,9.4,10,6.5,5.1,3.4,2,1.2,65.8,3.5,0.7,5.3,0,46.4,57.5,6.2,2.4,12.3,6.2,29.2,14,1.8,15.2,4.1,31.7,15.9,89.5,10.5,4,1.3,2.59,3.34,39547,2.49
4,1.861599,103689,16.5,17.7,16.6,14.8,13.3,10.3,5.9,3.2,2,50.5,8.4,9.1,8.7,7.7,6.8,5,2.7,1.3,0.7,49.5,8.1,8.7,7.9,7.1,6.5,5.3,3.2,1.9,1.2,44.4,3.3,0.9,4.4,0.1,87.8,86.4,10.4,4.8,19.9,9.5,9.8,4.5,1.4,5.3,3.2,60.1,26,94.5,5.5,1.3,1.2,4.60,4.63,23780,4.23
5,1.804747,92843,30,14.2,17.3,11.8,6.3,5.4,5.4,5.2,4.5,49.7,15.3,7.2,8.8,6.5,3.3,2.7,2.2,2,1.5,50.3,14.6,7,8.5,5.4,3,2.8,3.1,3.1,2.9,84.3,6.4,0.3,0.8,0,17.3,71.5,3.8,1.7,9.1,4.5,24.6,7.2,3.5,17.5,12.9,45.7,32.8,92.2,7.8,2.2,1.6,3.73,4.49,26337,4.16
6,1.505581,95397,18.7,19.6,16.5,13.8,12.9,9.4,4.8,2.6,1.4,50,9.6,10,8.6,6.8,6.4,4.7,2.3,1.1,0.5,50,9.3,9.5,8.1,7.1,6.4,4.7,2.6,1.5,0.9,48.6,4.8,1.2,1.6,0.4,81.7,83.7,9.9,5.2,19.2,11.2,12.3,5.7,1.6,6.6,3.7,60.7,20.1,93.9,6.1,3.1,1.3,4.18,4.42,24246,3.92
7,1.801882,93524,17.1,18.1,16.4,14.1,13.4,10.7,6.1,2.7,1.5,48.4,8.7,9.1,7.9,6.7,6.5,5.1,2.8,1.1,0.5,51.6,8.3,9,8.4,7.4,7,5.7,3.2,1.6,1,31.2,20.7,0.8,10.7,2.5,55.8,77.4,9,4.4,24.5,14.4,17.5,8.2,1.8,9.3,3.4,53,18.6,94.2,5.8,3.4,1,3.56,4.00,27646,3.33
8,1.020135,84641,16,15.4,15.9,14.3,13.5,12.5,7,3.4,1.9,49,8.1,7.8,8,7.1,6.6,6,3.3,1.5,0.7,51,7.9,7.5,7.9,7.3,7,6.6,3.8,1.9,1.2,37.8,16.1,0.9,14.2,0.9,45.6,76.5,7.6,3.6,18.8,10,17.4,7.4,1.6,9.9,3.7,46.5,20.3,92.3,7.7,2.9,2,3.27,3.68,27897,3.34
9,1.501437,88419,16.4,19.3,14.1,15.4,15.3,11,5.5,2.1,0.9,49.5,8.5,9.9,7,7.4,7.5,5.4,2.7,0.9,0.3,50.5,8,9.4,7,8,7.7,5.6,2.8,1.2,0.6,46.3,13.2,0.9,10.6,0.2,55.2,89.3,7.2,3.5,14,7.5,7,3.5,0.5,3.5,1.1,60.5,15.7,95.5,4.5,1,2,3.97,4.10,23271,4.30


In [39]:
minimerge = merged[0:50]
bestmodel = forward_selected(minimerge, 'pct_delta')
dir(bestmodel)

['HC0_se',
 'HC1_se',
 'HC2_se',
 'HC3_se',
 '_HCCM',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_data_attr',
 '_get_robustcov_results',
 '_is_nested',
 '_wexog_singular_values',
 'aic',
 'bic',
 'bse',
 'centered_tss',
 'compare_f_test',
 'compare_lm_test',
 'compare_lr_test',
 'condition_number',
 'conf_int',
 'conf_int_el',
 'cov_HC0',
 'cov_HC1',
 'cov_HC2',
 'cov_HC3',
 'cov_kwds',
 'cov_params',
 'cov_type',
 'df_model',
 'df_resid',
 'eigenvals',
 'el_test',
 'ess',
 'f_pvalue',
 'f_test',
 'fittedvalues',
 'fvalue',
 'get_influence',
 'get_robustcov_results',
 'initialize',
 'k_constant',
 'llf',
 'load',
 'model',
 'mse_model',
 'mse_resid',
 'mse_total',
 'nobs',
 'normalized_cov_params',
 'outlier_test',
 'params',
 'predict',
 'pvalues',


In [40]:
bestmodel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              pct_delta   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                     9527.
Date:                Sun, 06 Dec 2015   Prob (F-statistic):            0.00813
Time:                        15:49:48   Log-Likelihood:                 317.66
No. Observations:                  50   AIC:                            -537.3
Df Residuals:                       1   BIC:                            -443.6
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept        1.7458      0.003    586.040      0.001         1.708     1.784
A047[T.32.8]     0.4610      0.004    109.418      0.006         0.407     0.514
A047[T.33.0]    -0.2991      0.004    -70.999      0.009        -0.353    -0.246
A047[T.33.8]    -0.5119      0.004   -121.499      0.005        -0.565    -0.458
A047[T.36.5]    -0.0686      0.004    -16.278      0.039        -0.122    -0.015
A047[T.37.4]    -0.3835      0.004    -91.039      0.007        -0.437    -0.330
A047[T.38.6]    -0.2948      0.004    -69.968      0.009        -0.348    -0.241
A047[T.39.3]    -0.2218      0.004    -52.638      0.012        -0.275    -0.168
A047[T.39.8]    -0.7445      0.004   -176.725      0.004        -0.798    -0.691
A047[T.40.0]    -0.1870      0.004    -44.388      0.014        -0.241    -0.133
A047[T.40.2]    -0.3874      0.004    -91.949      0.007        -0.441    -0.334
A047[T.41.0]     0.1570      0.004     37.258      0.017         0.103     0.210
A047[T.41.9]    -0.7320      0.004   -173.761      0.004        -0.786    -0.678
A047[T.44.2]    -0.3423      0.004    -81.250      0.008        -0.396    -0.289
A047[T.44.4]    -0.5226      0.004   -143.246      0.004        -0.569    -0.476
A047[T.44.7]     0.3259      0.004     77.366      0.008         0.272     0.379
A047[T.45.3]    -0.0709      0.004    -16.835      0.038        -0.124    -0.017
A047[T.45.6]    -0.7154      0.004   -169.820      0.004        -0.769    -0.662
A047[T.45.7]     0.0590      0.004     14.001      0.045         0.005     0.113
A047[T.46.5]    -0.7256      0.004   -172.243      0.004        -0.779    -0.672
A047[T.47.1]    -0.3138      0.004    -74.497      0.009        -0.367    -0.260
A047[T.47.8]    -0.4193      0.004    -99.524      0.006        -0.473    -0.366
A047[T.48.0]    -0.5290      0.004   -125.579      0.005        -0.583    -0.476
A047[T.49.2]    -0.6425      0.004   -152.514      0.004        -0.696    -0.589
A047[T.49.6]    -0.2175      0.004    -51.634      0.012        -0.271    -0.164
A047[T.49.9]    -0.3608      0.004    -85.637      0.007        -0.414    -0.307
A047[T.50.2]    -0.2690      0.004    -63.857      0.010        -0.323    -0.215
A047[T.50.3]    -0.5999      0.004   -142.396      0.004        -0.653    -0.546
A047[T.50.4]     0.0142      0.004      3.382      0.183        -0.039     0.068
A047[T.50.9]    -0.2873      0.004    -68.208      0.009        -0.341    -0.234
A047[T.51.3]    -0.0354      0.004     -8.396      0.075        -0.089     0.018
A047[T.51.5]    -0.2419      0.004    -57.418      0.011        -0.295    -0.188
A047[T.51.6]    -0.3258      0.004    -77.342      0.008        -0.379    -0.272
A047[T.52.6]    -0.2917      0.004    -69.235      0.009        -0.345    -0.238
A047[T.53.0]     0.0561      0.004     13.320      0.048         0.003     0.110
A047[

In [55]:
response = 'pct_delta'
remaining = set(minimerge.columns)
remaining.remove(response)
formula = ''
for candidate in remaining:
    formula = formula + ' + ' + str(candidate)
formula = 'pct_delta ~ 1' + formula
formula

'pct_delta ~ 1 + A038 + A039 + A018 + A019 + A014 + A015 + A016 + A017 + A010 + A011 + A012 + A013 + A044 + A030 + A040 + A043 + A054 + A042 + A041 + A025 + A024 + A027 + A026 + A021 + A020 + A023 + A022 + A047 + A049 + A048 + A029 + A028 + A046 + A031 + A045 + A009 + A008 + A007 + A006 + A005 + A004 + A003 + A002 + A001 + A037 + A034 + A032 + A035 + A036 + A050 + A051 + A052 + A053 + A033 + A055 + A056'

In [62]:
regmodel = smf.ols(formula, minimerge).fit()

In [61]:
regmodel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              pct_delta   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                     0.000
Date:                Sun, 06 Dec 2015   Prob (F-statistic):                nan
Time:                        15:55:39   Log-Likelihood:                 1410.1
No. Observations:                  50   AIC:                            -2720.
Df Residuals:                       0   BIC:                            -2625.
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------------
Intercept                      0.0101        inf          0        nan           nan       nan
A038[T.3.8]                    0.0026        inf          0        nan           nan       nan
A038[T.4.0]                    0.0045        inf          0        nan           nan       nan
A038[T.4.5]                    0.0149        inf          0        nan           nan       nan
A038[T.4.7]                   -0.0073        inf         -0        nan           nan       nan
A038[T.5.1]                    0.0039        inf          0        nan           nan       nan
A038[T.5.2]                    0.0055        inf          0        nan           nan       nan
A038[T.5.5]                   -0.0107        inf         -0        nan           nan       nan
A038[T.5.7]                   -0.0052        inf         -0        nan           nan       nan
A038[T.5.8]                   -0.0035        inf         -0        nan           nan       nan
A038[T.6.2]                    0.0050        inf          0        nan           nan       nan
A038[T.6.3]                   -0.0026        inf         -0        nan           nan       nan
A038[T.6.5]                   -0.0046        inf         -0        nan           nan       nan
A038[T.6.8]                    0.0091        inf          0        nan           nan       nan
A038[T.6.9]                    0.0210        inf          0        nan           nan       nan
A038[T.7.0]                    0.0097        inf          0        nan           nan       nan
A038[T.7.2]                   -0.0021        inf         -0        nan           nan       nan
A038[T.7.3]                   -0.0031        inf         -0        nan           nan       nan
A038[T.7.4]                   -0.0026        inf         -0        nan           nan       nan
A038[T.7.6]                   -0.0047        inf         -0        nan           nan       nan
A038[T.8.1]                   -0.0047        inf         -0        nan           nan       nan
A038[T.8.2]                    0.0025        inf          0        nan           nan       nan
A038[T.8.4]                   -0.0154        inf         -0        nan           nan       nan
A038[T.8.5]                    0.0005        inf          0        nan           nan       nan
A038[T.8.6]                    0.0003        inf          0        nan           nan       nan
A038[T.8.7]                   -0.0122        inf         -0        nan           nan       nan
A038[T.8.8]                   -0.0007        inf         -0        nan           nan       nan
A038[T.9.0]                    0.0049        inf          0        nan           nan       nan
A038[T.9.4]                    0.0026        inf          0        nan           nan       nan
A038[T.9.5]                    0.0016      